In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/home/ajz55/Reddit/env/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# %matplotlib inline
# import itertools
# from wordcloud import WordCloud, STOPWORDS

# from src.features.preprocess import PreProcess

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.probability import FreqDist
# from nltk.corpus import stopwords, wordnet
# from nltk.stem import WordNetLemmatizer, PorterStemmer

# import re
# from gensim import corpora, models
# import gensim

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Read the posts and comments csv files

In [3]:
subreddit = "computerscience"
scrape_order = 'hot'

In [4]:
df_posts = pd.read_csv(f"../data/raw/{subreddit}_{scrape_order}_posts.csv")
df_posts.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created
0,n2n0ax,New to programming or computer science? Want a...,408,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09
1,qb4bof,THIS IS NOT A TECH SUPPORT SUB OR A COMPUTER R...,415,0.97,computerscience,https://www.reddit.com/r/computerscience/comme...,56,Tech Support: /r/techsupport\n\nComputer Recom...,1.634619e+09
2,skltie,Confusion Between Different Types of Optimizat...,10,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,2,I do not have a background in optimization and...,1.644002e+09
3,sk7puv,Behaviour of TCP and UDP traffic on the same n...,22,0.91,computerscience,https://www.reddit.com/r/computerscience/comme...,4,"So, as we all know , one of the main advantage...",1.643961e+09
4,skp5fm,Equivalent propositions to P=NP that are not c...,1,1.00,computerscience,https://www.reddit.com/r/computerscience/comme...,1,"Hello all,\n\nSo obviously, existence of a pol...",1.644011e+09


In [5]:
df_comments = pd.read_csv(f"../data/raw/{subreddit}_{scrape_order}_comments.csv")
df_comments.head()

,post_id,comment,comment_id,parent_id,created,is_submitter
0,n2n0ax,How late is too late to start a career in prog...,gwkmb17,t3_n2n0ax,1.619895e+09,False
1,n2n0ax,You’ll be 40 in a few years anyways. Why not t...,gwlbcha,t1_gwkmb17,1.619906e+09,False
2,n2n0ax,that is like the smartest thing Ive read in a ...,h06cu2y,t1_gwlbcha,1.622541e+09,False
3,n2n0ax,Hang in there!!!,h9d1o4m,t1_h06cu2y,1.629252e+09,False
4,n2n0ax,I'm so glad that I'm still seeing people repea...,h4oqtdr,t1_gwlbcha,1.625911e+09,False


In [6]:
df = df_posts.merge(df_comments, left_on='post_id', right_on='post_id', how='left')
df.head()

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created_x,comment,comment_id,parent_id,created_y,is_submitter
0,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,How late is too late to start a career in prog...,gwkmb17,t3_n2n0ax,1.619895e+09,False
1,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,You’ll be 40 in a few years anyways. Why not t...,gwlbcha,t1_gwkmb17,1.619906e+09,False
2,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,that is like the smartest thing Ive read in a ...,h06cu2y,t1_gwlbcha,1.622541e+09,False
3,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,Hang in there!!!,h9d1o4m,t1_h06cu2y,1.629252e+09,False
4,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,I'm so glad that I'm still seeing people repea...,h4oqtdr,t1_gwlbcha,1.625911e+09,False


In [7]:
# Check the number of posts with no comments
print("There are {} posts with no comments".format(df.comment.isna().sum()))
#print("There are {} posts with no content after filtering".format(len(df[df['comment'].str.len() == 0])))

There are 42 posts with no comments


# Import the bert model

In [8]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Traverse through each post

In [9]:
similarities = np.array([])
for post_id in df['post_id'].unique():
    temp = df[df['post_id'] == post_id]
    if pd.isnull(temp.iloc[0]['comment']):
        similarities = np.append(similarities, 0)
        continue
    title = temp.iloc[0]['title']
    body = temp.iloc[0]['body']
    
    topic = title
    # topic = title + body
    
    comments = temp['comment'].to_list()
    comments.append(topic)
    
    sentence_embeddings = model.encode(comments)
    similarity = cosine_similarity([sentence_embeddings[-1]], sentence_embeddings[:-1])
    similarities = np.append(similarities, similarity.flatten())

In [10]:
similarities

array([0.45664406, 0.45913619, 0.25022519, ..., 0.62445438, 0.42227167,
       0.42849961])

In [11]:
df['similarity'] = similarities

In [12]:
df

,post_id,title,score,upvote_ratio,subreddit,url,num_comments,body,created_x,comment,comment_id,parent_id,created_y,is_submitter,similarity
0,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,How late is too late to start a career in prog...,gwkmb17,t3_n2n0ax,1.619895e+09,False,0.456644
1,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,You’ll be 40 in a few years anyways. Why not t...,gwlbcha,t1_gwkmb17,1.619906e+09,False,0.459136
2,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,that is like the smartest thing Ive read in a ...,h06cu2y,t1_gwlbcha,1.622541e+09,False,0.250225
3,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,Hang in there!!!,h9d1o4m,t1_h06cu2y,1.629252e+09,False,0.200835
4,n2n0ax,New to programming or computer science? Want a...,408,1.0,computerscience,https://www.reddit.com/r/computerscience/comme...,1080,The previous thread was finally archived with ...,1.619890e+09,I'm so glad that I'm still seeing people repea...,h4oqtdr,t1_gwlbcha,1.625911e+09,False,0.398348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6791,q71moh,Can anyone learn to code well?,8,0.9,computerscience,https://www.reddit.com/r/computerscience/comme...,13,"Even though coding fascinates me, computer sci...",1.634092e+09,Anyone can do it. However not everyone can bui...,hgj7k5z,t3_q71moh,1.634160e+09,False,0.384200
6792,q71moh,Can anyone learn to code well?,8,0.9,computerscience,https://www.reddit.com/r/computerscience/comme...,13,"Even though coding fascinates me, computer sci...",1.634092e+09,Struggle is fine. Everyone struggles in someth...,hgjefoq,t3_q71moh,1.634163e+09,False,0.347361
6793,q71moh,Can anyone learn to code well?,8,0.9,computerscience,https://www.reddit.com/r/computerscience/comme...,13,"Even though coding fascinates me, computer sci...",1.634092e+09,A good resource for learning to code is defini...,hgmmu1k,t3_q71moh,1.634229e+09,False,0.624454
6794,q71moh,Can anyone learn to code well?,8,0.9,computerscience,https://www.reddit.com/r/computerscience/comme...,13,"Even though coding fascinates me, computer sci...",1.634092e+09,"No one is born a programmer. Sure, logic might...",hgv751f,t3_q71moh,1.634396e+09,False,0.422272


In [14]:
df.to_csv("../data/results/relevance_output.csv", index=False)

Notes: this takes a long time to run. Suggestion from another paper: use faiss.  
Also perhaps compare the comments with something else other than title

# Sample code below to generate embeddings and cosine similarity

In [6]:
sentence_embeddings = model.encode(df_posts['title'].to_list())

In [7]:
sentence_embeddings.shape

(495, 768)

In [8]:
sentence_embeddings

array([[ 0.30967617,  0.1629094 ,  1.3436381 , ..., -0.3773275 ,
        -0.74607503,  0.27187392],
       [ 0.6153924 ,  0.7614516 ,  1.5298136 , ...,  0.42983565,
        -0.63670594,  0.19039328],
       [-0.4176919 ,  0.29487124,  0.78496426, ..., -0.6611189 ,
        -0.96480453,  0.14001265],
       ...,
       [-0.5931168 ,  0.20732588, -0.35109794, ..., -0.72609115,
        -0.33478796,  0.01831246],
       [-0.20924684,  0.60646063,  0.4073564 , ...,  0.45895877,
         0.56324315, -0.05968758],
       [ 0.14121047,  0.04698378,  0.8630955 , ..., -0.47502777,
        -1.184771  ,  0.33819634]], dtype=float32)

In [ ]:
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)